In [4]:
#importing necessary libraries
import numpy as np
import torch
from sklearn.metrics import accuracy_score
import pickle

# Setting random seed for getting same accuracy matrix every time
seed = 45
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Function to load data
def load_data(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    features = data['features']   # storing features
    if 'targets' in data:
        targets = data['targets'] #storing targets
        targets = targets.clone().detach() if isinstance(targets, torch.Tensor) else torch.tensor(targets, dtype=torch.long)
    else:          # targets is None if data dont have targets
        targets = None
    return features, targets

# Function to print accuracy matrix
def print_accuracy_matrix(accuracy_matrix):
    n = len(accuracy_matrix)
    m = len(accuracy_matrix[0])
    for i in range(n):
        for j in range(i+1):
            print(f"{accuracy_matrix[i][j]:.2f}", end="\t")
        print()

# Initial LwP model training function using hard LWP model i.e each point belong to 1 class dont consider probability of belonging to other class
def train_lwp_hard(features, targets, regularization=1e-2, prototype_smoothing=0.2, noise_std=0.1):
    num_classes = 10
    prototypes = []  #storing prototype/mean of each class
    cov_matrices = []  #storing covariance of each class
    global_mean = features.mean(axis=0) # calculating global mean

    for i in range(num_classes):
        # Add Gaussian noise to the features of each class
        class_features = features[targets == i]
        noisy_class_features = class_features + np.random.normal(0, noise_std, class_features.shape)

        # smoothing the prototype with global mean
        prototype = (1 - prototype_smoothing) * noisy_class_features.mean(axis=0) + prototype_smoothing * global_mean
        prototypes.append(prototype)

        # covariance matrix regularizing
        cov_matrix = np.cov(noisy_class_features, rowvar=False) + regularization * np.eye(class_features.shape[1])
        cov_matrices.append(np.linalg.inv(cov_matrix))

    return np.array(prototypes), cov_matrices


# LwP model training function with soft clustering and progressive prototypes averaging i.e each points have probability of beloning to each class
def train_lwp_soft(features, prev_prototypes=None, prev_cov_matrices=None, alpha=0.5, num_classes=10):
    prototypes = []      #list to store prototypes/means        
    cov_matrices = []   #list to store covariance matrices

    # Compute soft assignments z_nk for each data point and each class

    soft_assignments = np.zeros((features.shape[0], num_classes))
    epsilon = 1e-12  # small value is taken to avoid division by zero error

    for k in range(num_classes): #calculating distances to each class
        distances = np.linalg.norm(features - prev_prototypes[k], axis=1) ** 2
        soft_assignments[:, k] = np.exp(-distances) 

    # add epsilon to soft_assignment_sum to avoid division by zero error
    soft_assignments_sum = soft_assignments.sum(axis=1, keepdims=True) + epsilon
    soft_assignments /= soft_assignments_sum #calcuate probability of belonging to each class     

    
    for k in range(num_classes): # Update prototype (mean) with soft assignments
        weighted_sum = np.sum(soft_assignments[:, k][:, np.newaxis] * features, axis=0)
        total_weight = np.sum(soft_assignments[:, k])

        # check whether total_weight > 0 or not
        if total_weight > 0:
            prototype = weighted_sum / total_weight
        else:
            # If total_weight is 0 use previous protype or use mean of features based on availability of prev_prototypes
            prototype = prev_prototypes[k] if prev_prototypes is not None else np.mean(features, axis=0)

        # update prototype based on given alpha value new prototype is weighted sum of current prototype and previous prototype
        prototype = alpha * prototype + (1 - alpha) * prev_prototypes[k]
        prototypes.append(prototype)

        # update covariance matrix with soft assignments
        cov_matrix = np.zeros((features.shape[1], features.shape[1]))
        for n in range(features.shape[0]):
            diff = (features[n] - prototype).reshape(-1, 1)
            cov_matrix += soft_assignments[n, k] * (diff @ diff.T)
        if total_weight > 0:
            cov_matrix /= total_weight
        else:
            cov_matrix = np.cov(features, rowvar=False) + np.eye(features.shape[1]) * 1e-6  # Use feature covariance as fallback

        # update covariance matrix based on given alpha values new covariance matrix is weighted sum of current covariance matrix and previous covariance matrix
        cov_matrix = alpha * cov_matrix + (1 - alpha) * np.linalg.inv(prev_cov_matrices[k])
        cov_matrices.append(np.linalg.inv(cov_matrix + np.eye(cov_matrix.shape[0]) * 1e-6))  #adding small value for stability purpose

    return np.array(prototypes), cov_matrices


#prediction function
def predict_lwp(features, prototypes, cov_matrices):
    predictions = []  # list to store predictions   
    for feature in features:
        distances = []  # list to store distance with respect to each class   
        for i in range(len(prototypes)):
            diff = feature - prototypes[i]
            mah_dist = np.sqrt(diff.T @ cov_matrices[i] @ diff)  # calculating mahalanobis distance
            distances.append(mah_dist)
        predicted_class = np.argmin(distances) #gives class which has less distance to the given feature
        predictions.append(predicted_class)
    return predictions  # returning predictions


#paths for the datasets
dataset_paths = [fr'/content/drive/MyDrive/M TECH/ML/mini-project-2/dataset/v/only_features/extracted_features_{i}_data.tar.pkl' for i in range(1, 11)]
heldout_paths = [fr'/content/drive/MyDrive/M TECH/ML/mini-project-2/dataset/v/extracted_features_{i}_data.tar.pkl' for i in range(1, 11)]


models = []  # a list to store means and covariance matrices for each model
accuracy_matrix = np.zeros((10, 10))  # a matrix to store accuracy of each model vs heldout datasets

# Training first model f1 on D1 using hard classification
features, targets = load_data(dataset_paths[0]) #loading features and targets
#get prototypes and covariance matrix for model 1 from train_lwp_hard function
prototypes, cov_matrices = train_lwp_hard(features, targets, regularization=1, prototype_smoothing=0.2)
models.append((prototypes, cov_matrices)) #appending means and covariance matrix to models list
print("Model f1 is completed")


#Updating models f2 to f10 using soft classification
for i in range(1, 10):
    features, _ = load_data(dataset_paths[i]) #loading features
    # get prototypes and covariance matrices for model i+1 from train_lwp_soft function
    prototypes, cov_matrices = train_lwp_soft(features, prev_prototypes=models[-1][0], prev_cov_matrices=models[-1][1], alpha=0.2)
    models.append((prototypes, cov_matrices)) #appending means and covariance matrix to models list
    print(f"Model f{i+1} is completed")

# Evaluation of models on held out datasets
for i, (prototypes, cov_matrices) in enumerate(models): #taking  model f1 to f10 iteratively
    for j in range(i + 1):   #taking data set 1 to data set i for calculating goodness of i th  model on them
        heldout_features, heldout_targets = load_data(heldout_paths[j]) #taking heldout features and targets
        heldout_predictions = predict_lwp(heldout_features, prototypes, cov_matrices) #get predictions from predict_lwp function
        accuracy = accuracy_score(heldout_targets.numpy(), heldout_predictions)
        accuracy_matrix[i, j] = accuracy * 100  # storing accuracy in terms of 0-100 range
    print(f"Evaluation of model{i+1} done!")

# Print accuracy matrix
print("Accuracy Matrix (Rows: Models f1 to f10, Columns: Held-out datasets D1 to D10):")
print_accuracy_matrix(accuracy_matrix)


Model f1 is completed
Model f2 is completed
Model f3 is completed
Model f4 is completed
Model f5 is completed
Model f6 is completed
Model f7 is completed
Model f8 is completed
Model f9 is completed
Model f10 is completed
Evaluation of model1 done!
Evaluation of model2 done!
Evaluation of model3 done!
Evaluation of model4 done!
Evaluation of model5 done!
Evaluation of model6 done!
Evaluation of model7 done!
Evaluation of model8 done!
Evaluation of model9 done!
Evaluation of model10 done!
Accuracy Matrix (Rows: Models f1 to f10, Columns: Held-out datasets D1 to D10):
99.96	
99.96	86.88	
99.96	86.96	87.32	
99.96	86.84	87.16	87.48	
99.96	86.80	87.32	87.40	87.44	
99.96	86.72	87.44	87.16	87.52	87.36	
99.96	86.72	87.40	87.16	87.48	87.40	86.24	
99.96	86.64	87.20	87.12	87.32	87.36	86.16	86.92	
99.96	86.64	87.28	87.20	87.44	87.32	86.20	86.88	86.12	
99.96	86.64	87.20	87.04	87.32	87.60	86.04	86.96	86.00	87.56	


In [6]:
# printing Confusion Matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(heldout_targets, heldout_predictions)
print(cm)

[[228   2   1   1   1   0   1   0   7   3]
 [  4 226   0   0   0   0   1   0   1  15]
 [ 10   1 202   6   6   4   4   1   0   0]
 [  2   1   3 211   5  31  16   0   0   1]
 [  2   1   5   6 189   1  12   3   0   0]
 [  2   0   0  27   2 212   2   6   0   0]
 [  0   0   8  15   3   6 220   0   0   0]
 [  2   2   1   6   7   8   1 211   1   2]
 [  9   6   0   1   0   0   0   0 264   4]
 [  5  20   2   3   0   0   0   0   2 226]]
